## Definitions



In [ ]:
import random
import itertools
import time

In [ ]:
#creates an edge
#adds reachablity for both vertices
#because it is an undirected graph
def add_edge(inputGraph, vertex, vertexTo):
    inputGraph[vertex].append(vertexTo)
    inputGraph[vertexTo].append(vertex)

#adds a vertex
def add_vertex(inputGraph, vertex):
    inputGraph[vertex] = []


def createRandomGraph(V, E):

    #check if ist is possible to create a random graph
    #with given vertex and edge numbers
    max = V*(V-1)/2
    if E>max:
      print("It is not a possible graph!")
      return

    #initilize the graph with adding all the vertices
    graph = {}
    for i in range(V):
        add_vertex(graph, i)
    
    #adds edges randomly
    for _ in range(E):
        selectRandomIndex = random.randint(0, V - 1)
        while len(graph[selectRandomIndex]) == V - 1 :
            selectRandomIndex = random.randint(0, V - 1)
        selectRandomIndex2 = random.randint(0, V - 1)
        while (selectRandomIndex2 == selectRandomIndex 
               or selectRandomIndex2 in graph[selectRandomIndex]):
            selectRandomIndex2 = random.randint(0, V - 1)
        add_edge(graph, selectRandomIndex, selectRandomIndex2)
    return graph

In [ ]:
def exactLongestCycle(graph):

    V = len(graph)
    vertex = list(range(V))
    longestCycle = []
    globalCycle = False

    #creates all the possible cycle permitations with a 
    #specific vertex number from bigger to smaller
    for size in range(V, 2, -1):
        allPermutations = list(itertools.permutations(vertex, size))

        #for every possible cycle
        for eachPermutation in allPermutations:
            eachPermutationList = list(eachPermutation)
            isCycle = True
            for index in range(len(eachPermutationList)):
                nextIndex = index + 1
                if index == len(eachPermutationList) - 1:
                    nextIndex = 0

                currentNode = eachPermutationList[index]
                nextNode = eachPermutationList[nextIndex]
                if nextNode not in graph[currentNode]:
                    isCycle = False
                    break

            #if this is really a cycle in the graph
            if isCycle:
                eachPermutationList.append(eachPermutationList[0])
                longestCycle = eachPermutationList.copy()
                globalCycle = True
                break
        if globalCycle:
            break

    return longestCycle

In [ ]:
def heuristicLongestCycle(graph):

    #initilize all vertices are not visited and parents are not determined.
    V = len(graph)
    visitedDict = {}
    parents = []
    for vertex in graph:
        visitedDict[vertex] = 0
        parents.append(-1)

    randomRootNodes = list(range(V))
    random.shuffle(randomRootNodes)
    cycleStartEnd = []
    isCycle = False

    #consider for all the vertices one by one
    for randomRoot in randomRootNodes:
        if visitedDict[randomRoot] == 0:
            #search for a cycle with using DFS
            isCycle = dfs(randomRoot, parents[randomRoot], parents, visitedDict, graph, cycleStartEnd)  
            #if it can find a cycle
            if isCycle: 
                #stop searching when it finds the first cycle
                break
    #forms the cycle array which is found by DFS
    if isCycle:
        cycleStart, cycleEnd = cycleStartEnd[0], cycleStartEnd[1]

        cycle = [cycleStart]
        v = cycleEnd

        while v != cycleStart:
            cycle.append(v)
            v = parents[v]

        cycle.append(cycleStart)
        return cycle
    #if no cycle is found
    else:
        return []

In [ ]:
#search for a cycle by DFS 
#when it finds the first cycle returns it
def dfs(v, p, parents, visitedNodes, graph, cycleStartEnd):
    visitedNodes[v] = 1  
    for neighbour in graph[v]:
        if neighbour != p:
            if visitedNodes[neighbour] == 0:
                parents[neighbour] = v
                if dfs(neighbour, v, parents, visitedNodes, graph, cycleStartEnd):
                    return True
            elif visitedNodes[neighbour] == 1:
                cycleStartEnd.append(neighbour)
                cycleStartEnd.append(v)
                return True

    visitedNodes[v] = 2
    return False

In [ ]:
def heuristicLongestCycle2(graph):

  #initilize all vertices are not visited and parents are not determined.
  V = len(graph)
  visitedDict = {}
  parents = []
  for vertex in graph:
    visitedDict[vertex] = 0
    parents.append(-1)
    
  randomRootNodes = list(range(V))
  random.shuffle(randomRootNodes)
  cycleStartEnd = []
  isCycle = False

  pos_results = []

  for _ in range(10): 
    #consider for all the vertices one by one
    for randomRoot in randomRootNodes:
      if visitedDict[randomRoot] == 0:
        #search for a cycle with using DFS
        isCycle = dfs(randomRoot, parents[randomRoot], parents, visitedDict, graph, cycleStartEnd)
        #if it can find a cycle
        if isCycle:  
          #forms the cycle array which is found by DFS 
          cycleStart, cycleEnd = cycleStartEnd[0], cycleStartEnd[1]
          cycle = [cycleStart]
          v = cycleEnd
          while v != cycleStart:
            cycle.append(v)
            v = parents[v]
          cycle.append(cycleStart)
          #adds the found cycle into possible results array
          pos_results.append(cycle.copy())
          cycle.clear()
          cycleStartEnd.clear()
          parents.clear()
          for vertex in graph:
            visitedDict[vertex] = 0
            parents.append(-1)
          #it continues to search cycles but starting from other vertices

  #finds the maximum result from all the possible results found
  if len(pos_results)!=0:
    max = 0
    res = []
    for i in pos_results:
      if len(i)>max:
        res = i
        max = len(i)
    #returns the longest cycle from found cycles
    return res
  #if no cycle is found
  else:
    return []

In [ ]:
def improveCycle1(graph, result):

    #for every vertices in the result cycle
    for i in range(len(result) - 2):  

        neighbourList1 = graph[result[i]] 
        neighbourList2 = graph[result[i + 1]]
        #finds the intersection of neighbour vertices of ith and (i+1)th
        #vertices in the result cycle
        sharedNodes = list(set(neighbourList1).intersection(neighbourList2))

        #for every common vertices
        if sharedNodes:
            for sharedNode in sharedNodes:
                #if it is not already in the result
                if sharedNode not in result:
                  #add the new vertex between ith and (i+1)th vertices 
                  #in result cycle
                  result.insert(i+1, sharedNode)
                  break

    return result

In [ ]:
def improveCycle2(graph, result):

  #choose a random number
  V = len(graph)
  iter_num = random.randint(3,V-1)
  rand_list = []

  #forms a index list whose length is the previous
  #random number. The index numbers are  also
  #choosen randomly 
  for i in range(len(result) - 2):
    neighbourList = graph[result[i]]
    if len(neighbourList) > 2:
      if len(neighbourList) < iter_num:
        for a in range(len(neighbourList)):
          rand_num = random.randint(0, len(neighbourList)-1) 
          rand_list.append(rand_num)
      else:
        for a in range(iter_num):
          rand_num = random.randint(0, len(neighbourList)-1) 
          rand_list.append(rand_num)
    checker = False

    #for some neighbour vertices of the ith vertex in result
    #(these ares choosen randomly with the random array created previously)
    for index in rand_list:
      #if it is not already in the result
      if (neighbourList[index] not in result):
        neighbourList1 = graph[ neighbourList[index] ] 
        if i+1 < len(result):
          neighbourList2 = graph[ result[i + 1] ]
          #find its common neighbour with (i+1)th vertex from result
          sharedNodes = list(set(neighbourList1).intersection(neighbourList2))
          if sharedNodes:
            for sharedNode in sharedNodes:
              #if it is not already in the result
              if sharedNode not in result:
                #add the new 2 vertices between ith and (i+1)th vertices
                #in result cycle
                result.insert(i+1, neighbourList[index])
                result.insert(i+2, sharedNode)
                checker = True
                break
      if checker:
        break
    rand_list.clear()


  return result


In [ ]:
def improveCycle3(graph, result):

  #choose a random number
  V = len(graph)
  iter_num = random.randint(3,V-1)
  rand_list = []

  #forms a index list whose length is the previous
  #random number. The index numbers are  also
  #choosen randomly
  for i in range(len(result) - 3):
    neighbourList = graph[result[i]]
    if len(neighbourList) > 2:
      if len(neighbourList) < iter_num:
        for a in range(len(neighbourList)):
          rand_num = random.randint(0, len(neighbourList)-1) 
          while rand_num in rand_list:
            rand_num = random.randint(0, len(neighbourList)-1) 
          rand_list.append(rand_num)
      else:
        for a in range(iter_num):
          rand_num = random.randint(0, len(neighbourList)-1) 
          while rand_num in rand_list:
            rand_num = random.randint(0, len(neighbourList)-1)
          rand_list.append(rand_num)
    checker = False

    #for some neighbour vertices of the ith vertex in result
    #(these ares choosen randomly with the random array created previously)
    for index in rand_list:
      #if it is not already in the result
      if (neighbourList[index] not in result):
        neighbourList1 = graph[ neighbourList[index] ] 
        if i+2 < len(result):
          neighbourList2 = graph[ result[i + 2] ]
          #find its common neighbour with (i+2)th vertex from result
          sharedNodes = list(set(neighbourList1).intersection(neighbourList2))
          if sharedNodes:
            for sharedNode in sharedNodes:
              #if it is not already in the result
              if sharedNode not in result:
                #add the new 2 vertices between ith and (i+2)th vertices
                #in result cycle and pops the (i+1)th vertex from result
                result.pop(i+1)
                result.insert(i+1, neighbourList[index])
                result.insert(i+2, sharedNode)
                checker = True
                break
      if checker:
        break
    rand_list.clear()

  return result

In [ ]:
def improveCycleUltimate(graph, result):

  #it only runs all the improve cycle
  #algorithms one after another
  improveCycle1(graph, result)
  improveCycle2(graph, result)
  improveCycle3(graph, result)

  return result

In [ ]:
def correctness(graph,result):

  #if it is empty
  if len(result) == 0:
    return True

  if len(result)>2:
    #if first and last vertes is not same
    if result[0] != result[-1]:
      return False

    #check for every consecutive vertex to 
    #wheter it can reach the next vertex
    #or not in the graph
    for idx in range(len(result)-1):
      if result[idx+1] not in graph[result[idx]]:
        return False
    
    return True
  #if length is 1 or 2
  else:
    return  False